In [1]:
%cd ../../..

/workspaces/Code


In [2]:
from __future__ import annotations

from dataclasses import dataclass
import numpy as np
import tensorflow as tf
import math
import os
import gc
from datetime import datetime
import json
import statistics
from typing import Iterator
from buslane.commands import CommandHandler
from itertools import product
import multiprocessing


from tensorflow.keras import mixed_precision

from image_provider import ImageProvider
from test_runner import TestRunner, RepeatResult, DroneResult, TestRun
from map_provider import MapProvider, ImageProjection
from DroneProvider import SimulatedDroneProvider
from vector import Vector2D
from map_plotter import MapPlotter
from WeightCalculators.Transformers import StretchedTanTransformer
from WeightCalculators import ModelBasedWeightCalculator
from know_location_drone_localizer import KnownLocationDroneLocalizer, StepCommand

from ModelBuilders import TestEfficientNetB0Builder, TestMobileNetBuilder, TestVgg16NetBuilder, TestEfficientNetB2Builder, TestResNetBuilder, TestBaseModelBuilder, ModelOptions

In [ ]:
def enable_gpu_memory_growth():
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
    policy = mixed_precision.Policy('mixed_float16')
    mixed_precision.set_global_policy(policy)

    """
    Enables memory growth mode for GPUs.
    """
    gpus = tf.config.experimental.list_physical_devices('GPU')
    assert len(gpus) > 0, "No GPUs detected!"
            
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
crop_size = 672
accuracy_threshold = 25
particle_counts = [1500, 1000, 500, 250, 150, 100, 75]
particle_randomize_percent = 0

drone_move_noise = 30
drone_last_known_noise = 50
initial_location_offset = 25

repeat_count = 10
drone_steps = 25
graph_size = 1200

In [ ]:
def get_distance_at_45d(distance: int) -> int:
    coord = math.sqrt(math.pow(distance, 2) / 2)
    return math.floor(coord)

def build_drones(crop_size: int) -> Iterator[TestRun]:
    #6024x6024
    city_image = ImageProvider("City/NewTraining/ExperimentZone/City_2017.jpg")
    drone_image = ImageProvider("City/NewTraining/ExperimentZone/City_2016.jpg")

    projection = ImageProjection(Vector2D(0, 524), Vector2D(5000, 5000))
    city_map = MapProvider(
        image_provider=city_image,
        crop_size=crop_size,
        projection=projection)
    
    drone_map = MapProvider(
        image_provider=drone_image,
        crop_size=crop_size,
        projection=projection)
    
    run_name = "Forest"
    for move_by in [60, 100]:
        yield TestRun(run_name, SimulatedDroneProvider(drone_map, Vector2D(1000, 2500), Vector2D(move_by, 0), drone_steps), city_map)

    projection = ImageProjection(Vector2D(524, 0), Vector2D(5000, 5000))
    city_map = MapProvider(
        image_provider=city_image,
        crop_size=crop_size,
        projection=projection)
    
    drone_map = MapProvider(
        image_provider=drone_image,
        crop_size=crop_size,
        projection=projection)
    
    run_name = "Residential"
    for move_by in [60, 100]:
        move_by_side = get_distance_at_45d(move_by)
        yield TestRun(run_name, SimulatedDroneProvider(drone_map, Vector2D(2500, 1000), Vector2D(move_by_side, move_by_side), drone_steps), city_map)

    projection = ImageProjection(Vector2D(0, 1000), Vector2D(5000, 5000))
    city_map = MapProvider(
        image_provider=city_image,
        crop_size=crop_size,
        projection=projection)
    
    drone_map = MapProvider(
        image_provider=drone_image,
        crop_size=crop_size,
        projection=projection)
    
    run_name = "Apartments"
    for move_by in [60, 100]:
        yield TestRun(run_name, SimulatedDroneProvider(drone_map, Vector2D(2500, 4000), Vector2D(0, -move_by), drone_steps), city_map)

In [ ]:
def _build_drone_folder(base_folder: str, run_name: str, drone_repr: str):
    drone_dir = f"{base_folder}/{run_name}/drone-{drone_repr}"
    os.makedirs(drone_dir, exist_ok=True)

    return drone_dir

def _build_repeat_folder(base_folder: str, run_name: str, drone_repr: str, repeat_index: int):
    drone_folder = _build_drone_folder(base_folder, run_name, drone_repr)

    repeat_dir = f"{drone_folder}/repeat-{repeat_index:02d}"
    os.makedirs(repeat_dir, exist_ok=True)

    return repeat_dir

In [ ]:
class StepCommandHandler(CommandHandler[StepCommand]):
    def __init__(self, base_folder: str, run_name: str, plotter: MapPlotter):
        self.base_folder = base_folder
        self.run_name = run_name

        self.plotter = plotter

    def handle(self, command: StepCommand) -> None:
        drone = command.drone

        # fig = self.plotter.plot_graph(
        #     particles=command.particles,
        #     weights=command.weights,
        #     title="Drono lokalizavimas",
        #     drone_x=drone.get_position().x,
        #     drone_y=drone.get_position().y,
        #     predicted_x=command.probable_position.x,
        #     predicted_y=command.probable_position.y)

        # fig.show()

        prediction_distance = drone.get_position().distance_to(command.probable_position)
        print(f"\t\t Flying... Step {drone.get_current_step()}. Distance: {prediction_distance}")

        # folder = _build_repeat_folder(self.base_folder, self.run_name, f"{drone}", drone.get_reset_count())
        # filename = f"{folder}/step_{drone.get_current_step():02d}.jpg"

        # fig.write_image(filename, engine="kaleido")

class RepeatResultHandler(CommandHandler[RepeatResult]):
    def __init__(self, base_folder: str, run_name: str,):
        self.base_folder = base_folder
        self.run_name = run_name

    def handle(self, command: RepeatResult) -> None:
        repeat_folder = _build_repeat_folder(self.base_folder, self.run_name, command.drone_repr, command.repeat_index)
        repeat_file = f"{repeat_folder}/repeat-stats.json"

        with open(repeat_file, "w") as file:
            file.write(json.dumps(command.dump(), indent=4, sort_keys=True))

class DroneResultHandler(CommandHandler[DroneResult]):
    def __init__(self, base_folder: str, run_name: str,):
        self.base_folder = base_folder
        self.run_name = run_name

    def handle(self, command: DroneResult) -> None:
        drone_folder = _build_drone_folder(self.base_folder, self.run_name, command.drone_repr)
        drone_file = f"{drone_folder}/route-stats.json"

        with open(drone_file, "w") as file:
            file.write(json.dumps(command.dump(), indent=4, sort_keys=True))

#### Run FUNCTION

In [ ]:
class ParticleFilterRunner:
    def __init__(self, model_builder: TestBaseModelBuilder, similarity_threshold: float, particle_count: int):
        self.model_builder = model_builder
        self.similarity_threshold = similarity_threshold
        self.particle_count = particle_count

        base_folder = f"Data/ParticleFilter/BestOf/{crop_size}/Particles-{particle_count}"

        builder_options = model_builder.get_options()
        builder_label = builder_options.builder_label
        representation = builder_options.representation()

        current_date = datetime.now().strftime("%Y-%m-%d.%H-%M-%S")
        self.run_folder = f"{base_folder}/{builder_label}/{representation}/{current_date}"

    def _get_distances(self, results: list[DroneResult]):
        distances = []
        for result in results:
            for repeat in result.repeat_results:
                for step in repeat.step_results:
                    distances.append(step.distance)

        return distances

    def save_all_runs(self, results: list[DroneResult]):
        run_data = {
            "average_acc": statistics.mean([result.average_accuracy for result in results]),
            "average_distance": statistics.mean(self._get_distances(results)),
            "distance_std": statistics.stdev(self._get_distances(results)),
            "max_distance": max(self._get_distances(results)),
            "min_distance": min(self._get_distances(results)),
            "runs": [
                {
                    "drone": result.drone_repr,
                    "average_accuracy": result.average_accuracy, 
                    "average_distance": result.avg_distances(),
                    "distance_std": result.distances_std(),
                    "min_distance": min(result.get_distances()),
                    "max_distance": max(result.get_distances()),
                }
                for result in results
            ],
        }

        run_file = f"{self.run_folder}/run-stats.json"
        with open(run_file, "w") as file:
            file.write(json.dumps(run_data, indent=4))

    def run(self):
        model_calculator = ModelBasedWeightCalculator(
            self.model_builder,
            batch_size=48,
            transformer=StretchedTanTransformer(self.similarity_threshold))

        localizer = KnownLocationDroneLocalizer(
            particle_count=self.particle_count,
            weight_calculator=model_calculator,
            particle_randomize_percent=particle_randomize_percent,
            drone_move_noise=drone_move_noise,
            drone_last_known_noise=drone_last_known_noise,
            initial_location_offset=initial_location_offset)

        runner = TestRunner(localizer, accuracy_threshold, repeat_count=repeat_count)

        test_runs = build_drones(crop_size=crop_size)

        results: list[DroneResult] = []
        for test_run in test_runs:
            name, drone, city_map = test_run.name, test_run.drone, test_run.city_map

            plotter = MapPlotter(city_map, graph_size=graph_size)

            runner.command_bus.clear()
            runner.command_bus.register(handler=RepeatResultHandler(self.run_folder, name))

            localizer.command_bus.clear()
            localizer.command_bus.register(handler=StepCommandHandler(self.run_folder, name, plotter=plotter))

            print(f"Starting Drone Route: {drone}")
            drone_result = runner.run(test_run)
            print(f"Ended Drone Route: {drone}. Average accuracy: {drone_result.average_accuracy}")

            results.append(drone_result)
            DroneResultHandler(self.run_folder, name).handle(drone_result)

        self.save_all_runs(results)


In [ ]:
@dataclass
class ModelNN:
    modelNN: int
    trainable_froms: list[int]

@dataclass
class BuilderTests:
    def __init__(self, 
        ingest_denses: list[int],
        output_denses: list[int],
        modelNNs: list[ModelNN],
        epochs_list: list[int],
        batch_list: list[int],
        builder_label: str):

        self.ingest_denses = ingest_denses
        self.output_denses = output_denses
        self.modelNNs = modelNNs
        self.epochs_list = epochs_list
        self.batch_list = batch_list

        self.builder_label = builder_label
        
    def generate_options(self):
        tests = product(self.ingest_denses, self.output_denses, self.modelNNs, self.epochs_list, self.batch_list)

        for ingest_dense, output_dense, modellNN, epochs, batch_size in tests:
            for trainable_from in modellNN.trainable_froms:
                if trainable_from > modellNN.modelNN:
                    continue

                yield ModelOptions(
                    builder_label=self.builder_label,
                    model_nn=modellNN.modelNN,
                    ingest_dense=ingest_dense,
                    output_dense=output_dense,
                    trainable_from_index=trainable_from,
                    epochs=epochs,
                    batch_size=batch_size)


In [ ]:
effb0_similarities = [0.662, 0.763, 0.654, 0.651, 0.722]
efficient_b0_tests = BuilderTests(
    ingest_denses=[64],
    output_denses=[8],
    modelNNs=[
        ModelNN(30, [0, 30]),
        ModelNN(71, [30]),
        ModelNN(140, [140]),
        ModelNN(254, [254]),
    ],
    epochs_list=[6],
    batch_list=[16],
    builder_label="Experimental/NewTraining/EfficientNetV2B0")


for test in efficient_b0_tests.generate_options():
    print(test.representation())

def run_efficient_net(options, similarity, particle_count):
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

    enable_gpu_memory_growth()

    model_builder = TestEfficientNetB0Builder(options)
    particle_filter = ParticleFilterRunner(model_builder, similarity, particle_count)

    particle_filter.run()

    gc.collect()
    print("-" * 80)

In [ ]:
for options, similarity in zip(efficient_b0_tests.generate_options(), effb0_similarities):
    for particle_count in particle_counts:
        p1 = multiprocessing.Process(target=run_efficient_net, args=[options, similarity, particle_count])

        p1.start()
        p1.join()

In [ ]:
effb2_similarities = [0.721, 0.84, 0.722, 0.697, 0.944, 0.861]
efficient_b2_tests = BuilderTests(
    ingest_denses=[64],
    output_denses=[8],
    modelNNs=[
        ModelNN(59, [59, 25]),
        ModelNN(68, [25, 0]),
        ModelNN(111, [111]),
        ModelNN(331, [331]),
    ],
    epochs_list=[6],
    batch_list=[16],
    builder_label="Experimental/NewTraining/EfficientNetB2")

for test in efficient_b2_tests.generate_options():
    print(test.representation())

def run_efficientb2_net(options, similarity, particle_count):
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
    enable_gpu_memory_growth()

    model_builder = TestEfficientNetB2Builder(options)
    particle_filter = ParticleFilterRunner(model_builder, similarity, particle_count)

    particle_filter.run()

    gc.collect()
    print("-" * 80)

In [ ]:
for options, similarity in zip(efficient_b2_tests.generate_options(), effb2_similarities):
    for particle_count in particle_counts:
        p1 = multiprocessing.Process(target=run_efficientb2_net, args=[options, similarity, particle_count])

        p1.start()
        p1.join()

In [ ]:
vgg_similarities = [0.699, 0.704, 0.728]
vgg_tests = BuilderTests(
    ingest_denses=[64],
    output_denses=[8],
    modelNNs=[
        ModelNN(10, [10]),
        ModelNN(14, [14]),
        ModelNN(18, [18]),
    ],
    epochs_list=[6],
    batch_list=[4],
    builder_label="Experimental/NewTraining/VGG16")

for test in vgg_tests.generate_options():
    print(test.representation())

def run_vgg16_net(options, similarity, particle_count):
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
    enable_gpu_memory_growth()

    model_builder = TestVgg16NetBuilder(options)
    particle_filter = ParticleFilterRunner(model_builder, similarity, particle_count)

    particle_filter.run()

    gc.collect()
    print("-" * 80)

In [ ]:
for options, similarity in zip(vgg_tests.generate_options(), vgg_similarities):
    for particle_count in particle_counts:
        p1 = multiprocessing.Process(target=run_vgg16_net, args=[options, similarity, particle_count])

        p1.start()
        p1.join()

In [ ]:
resnet_similarities = [0.735, 0.78, 0.719, 0.66, 0.719]
resnet_tests = BuilderTests(
    ingest_denses=[128],
    output_denses=[8],
    modelNNs=[
        ModelNN(50, [0, 50]),  
        ModelNN(80, [0, 38]),
        ModelNN(142, [142]),
    ],
    epochs_list=[6],
    batch_list=[16],
    builder_label="Experimental/NewTraining/ResNet50")

for test in resnet_tests.generate_options():
    print(test.representation())

def run_resnet_net(options, similarity, particle_count):
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

    enable_gpu_memory_growth()

    model_builder = TestResNetBuilder(options)
    particle_filter = ParticleFilterRunner(model_builder, similarity, particle_count)

    particle_filter.run()

    gc.collect()
    print("-" * 80)

In [ ]:
for options, similarity in zip(resnet_tests.generate_options(), resnet_similarities):
    for particle_count in particle_counts:
        p1 = multiprocessing.Process(target=run_resnet_net, args=[options, similarity, particle_count])

        p1.start()
        p1.join()

In [ ]:
mobile_similarities = [0.691, 0.853, 0.688, 0.692, 0.7, 0.841]
mobile_tests = BuilderTests(
    ingest_denses=[64],
    output_denses=[8],
    modelNNs=[
        ModelNN(35, [0, 35]),
        ModelNN(54, [0, 35]),
        ModelNN(72, [0, 72]),
    ],
    epochs_list=[6],
    batch_list=[16],
    builder_label="Experimental/NewTraining/MobileNet")

for test in mobile_tests.generate_options():
    print(test.representation())

def run_mobile_net(options, similarity, particle_count):
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
    enable_gpu_memory_growth()

    model_builder = TestMobileNetBuilder(options)
    particle_filter = ParticleFilterRunner(model_builder, similarity, particle_count)

    particle_filter.run()

    gc.collect()
    print("-" * 80)

In [ ]:
for options, similarity in zip(mobile_tests.generate_options(), mobile_similarities):
    for particle_count in particle_counts:
        p1 = multiprocessing.Process(target=run_mobile_net, args=[options, similarity, particle_count])

        p1.start()
        p1.join()